In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tab_transformer_pytorch import TabTransformer, FTTransformer
from preprocessing import get_features_and_target
from sklearn.preprocessing import LabelEncoder
from RMSELoss import RMSELoss
import plotly.graph_objects as go

# Get Dataset

In [2]:
train_df = pd.read_csv("data/train_data.csv")
dev_df = pd.read_csv("data/development_data.csv")

target_column = "PullTest (N)"  

x_train, y_train = get_features_and_target(train_df, target_column)
x_dev, y_dev = get_features_and_target(dev_df, target_column)


# Augment for New Columns

In [3]:
# Augment the training DataFrame with empty columns for calculations

train_df_c = train_df
train_df_c['FPull_4d_310MPa'] = ''
train_df_c['FPull_5d_310MPa'] = ''
train_df_c['FPull_4d_365MPa'] = ''
train_df_c['FPull_5d_365MPa'] = ''
train_df_c['FPull_4d_440MPa'] = ''
train_df_c['FPull_5d_440MPa'] = ''

# Calculate values

Formula: 

Shear-failure model: 

F = A * τ 

A = pi * d² / 4

τ ≈ 0.6 * σ

Nugget Diameter Relation:

d ≈ k * √t           (with k = 4 or 5)

# F = (pi / 4) * ((4 or 5) * √t)² * (0.6 * σ)

In [16]:
for i in range(train_df_c.shape[0]):

    # Calculate the thickness based on the minimum of either Thickness A or Thickness B
    if train_df_c['Thickness A (mm)'][i] <= train_df_c['Thickness B (mm)'][i]:
        t = train_df_c['Thickness A (mm)'][i]
    else:
        t = train_df_c['Thickness B (mm)'][i]

    # Calculate the pull force for different diameters and yield strengths
    f_pull_4d_310MPa = (np.pi/4) * np.square(4 * np.sqrt(t)) * (0.8 * 310)
    train_df_c.loc[i, 'FPull_4d_310MPa'] = round(f_pull_4d_310MPa, 1)
    f_pull_5d_310MPa = (np.pi/4) * np.square(5 * np.sqrt(t)) * (0.8 * 310)
    train_df_c.loc[i, 'FPull_5d_310MPa'] = round(f_pull_5d_310MPa, 1)

    f_pull_4d_365MPa = (np.pi/4) * np.square(4 * np.sqrt(t)) * (0.8 * 365)
    train_df_c.loc[i, 'FPull_4d_365MPa'] = round(f_pull_4d_365MPa, 1)
    f_pull_5d_365MPa = (np.pi/4) * np.square(5 * np.sqrt(t)) * (0.8 * 365)
    train_df_c.loc[i, 'FPull_5d_365MPa'] = round(f_pull_5d_365MPa, 1)

    f_pull_4d_310MPa = (np.pi/4) * np.square(4 * np.sqrt(t)) * (0.8 * 440)
    train_df_c.loc[i, 'FPull_4d_440MPa'] = round(f_pull_4d_310MPa, 1)
    f_pull_5d_310MPa = (np.pi/4) * np.square(5 * np.sqrt(t)) * (0.8 * 440)
    train_df_c.loc[i, 'FPull_5d_440MPa'] = round(f_pull_5d_310MPa, 1)

# About (0.6·σ)

Different sources mention different values for the 0,6. If you go with Tresca or von Mises criterion you get 0,5-0,57 which gets approximated to 0,6.  With 45 degree shear plane theroy the relationship is 1/√2 which is 0,707. It is also typical for 0,5 to be used. austenitic stainless steels can range up to ~ 0.8 this material is low carbon steel though and doesn't fall into this category.

# Check Dataset

In [5]:
train_df_c.head()

,Sample ID,Pressure (PSI),Welding Time (ms),Angle (Deg),Force (N),Current (A),Thickness A (mm),Thickness B (mm),Material,PullTest (N),NuggetDiameter (mm),Category,Comments,Thickness A+B (mm),FPull_4d_310MPa,FPull_5d_310MPa,FPull_4d_365MPa,FPull_5d_365MPa,FPull_4d_440MPa,FPull_5d_440MPa
0,101,80,800,0,115.54,2929.03,0.625,0.631,AISI 1010 carbon steel,2683.1,3.41,Good,NaN,1.256,1460.8,2282.6,1720.0,2687.5,2073.5,3239.8
1,31,95,200,15,8.31,1075.09,0.946,0.939,AISI 1010 carbon steel,2667.3,2.81,Bad,DOE,1.885,2194.8,3429.3,2584.2,4037.8,3115.2,4867.4
2,493,60,1200,0,97.09,4161.74,0.615,0.619,AISI 1010 carbon steel,2730.3,3.58,Good,NaN,1.234,1437.5,2246.0,1692.5,2644.5,2040.3,3187.9
3,219,60,400,0,93.01,3140.98,0.621,0.632,AISI 1010 carbon steel,2748.3,3.38,Good,NaN,1.253,1451.5,2268.0,1709.0,2670.3,2060.2,3219.0
4,60,80,1000,0,96.29,3615.49,0.631,0.640,AISI 1010 carbon steel,2946.9,3.73,Good,NaN,1.271,1474.9,2304.5,1736.5,2713.3,2093.4,3270.9


# Graph

In [17]:
x = train_df_c.index

fig = go.Figure()

groups = {
    "310MPa": {
        "cols": ["FPull_4d_310MPa", "FPull_5d_310MPa"],
        "colors": ["lightblue", "blue"]
    },
    "365MPa": {
        "cols": ["FPull_4d_365MPa", "FPull_5d_365MPa"],
        "colors": ["lightgreen", "green"]
    },
    "440MPa": {
        "cols": ["FPull_4d_440MPa", "FPull_5d_440MPa"],
        "colors": ["lightcoral", "red"]
    }
}

for info in groups.values():
    for col, col_color in zip(info["cols"], info["colors"]):
        fig.add_trace(
            go.Scatter(
                x=x,
                y=train_df_c[col],
                mode="lines+markers",
                name=col,
                line=dict(color=col_color, width=2),
                marker=dict(color=col_color)
            )
        )

fig.add_trace(
    go.Scatter(
        x=x,
        y=train_df_c["PullTest (N)"],
        mode="lines+markers",
        name="PullTest (N)",
        line=dict(color="grey", width=2, dash="dash"),
        marker=dict(color="grey", size=4)
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=train_df_c["NuggetDiameter (mm)"],
        mode="lines+markers",
        name="NuggetDiameter (mm)",
        line=dict(color="black", width=2),
        marker=dict(color="black", size=4),
        yaxis="y2"
    )
)

fig.update_layout(
    title="Pull Force and Nugget Diameter Over Different MPa Values and Nugget Diameter Recommendations",
    xaxis_title="Row Number",
    yaxis=dict(
        title="Force (N) or MPa",
        showgrid=True
    ),
    yaxis2=dict(
        title="Nugget Diameter (mm)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend_title="Series",
    template="seaborn",
    legend=dict(
        x=1.1,        # push legend past the right edge
        y=1,
        xanchor="left",
        borderwidth=1
    ),
    margin=dict(r=200)  # give extra room on right for the legend
)

fig.show()